In [1]:
from deltalake import DeltaTable

dt = DeltaTable("s3://delta/iot_beacon_ingest_report_v1", None, {
    "aws_default_region": "us-east-2",
    "aws_secret_access_key": "minioadmin",
    "allow_http": "true",
    "aws_access_key_id": "minioadmin",
    "aws_endpoint": "http://localhost:9000"
})



In [3]:
dt.files()

['date=1970-01-01/part-00000-0a2e8608-fdff-4375-9443-36b956112ed7-c000.snappy.parquet']

In [5]:
dt.to_pandas()

,received_timestamp,report,date
0,1677136697837,{'pub_key': b'\x00F\xa9L\x04\x0f\xc5\xe8>\x819...,1970-01-01
1,1677136698034,{'pub_key': b'\x00d\xe4\xef\xc6\xcb 4\xedD\xb8...,1970-01-01
2,1677136698215,{'pub_key': b'\x00\xdc\xdcD3\xb2\x0f8\xa0Gv\xc...,1970-01-01
3,1677136698345,{'pub_key': b'\x00\x10\xe8G\x12P\xfe\xb8\x1d\x...,1970-01-01
4,1677136698376,{'pub_key': b'\x00w#\xe0\xd2tn\xe8[A\x97\xeda\...,1970-01-01
...,...,...,...
7545,1677197117133,"{'pub_key': b""\x00\xae\xf4\x1byc\xec\xc0Q\x90K...",1970-01-01
7546,1677197117154,{'pub_key': b'\x00E\x02\xaf\x8d\xa8\t\x01@9\xb...,1970-01-01
7547,1677197117225,{'pub_key': b'\x00\xe5\xe9\xf4]\xadS\xf1eo\x10...,1970-01-01
7548,1677197117266,{'pub_key': b'\x00\xf5\xb0\xd1\x96\xf8p\xe0n\x...,1970-01-01
